In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Softmax

In [2]:
df = pd.read_csv('train.csv')
df.head()
df.pop('id')
df.dtypes

feature_0      int64
feature_1      int64
feature_2      int64
feature_3      int64
feature_4      int64
               ...  
feature_71     int64
feature_72     int64
feature_73     int64
feature_74     int64
target        object
Length: 76, dtype: object

In [3]:
target = df.pop('target').tolist()

In [4]:
test_df = pd.read_csv('test.csv')
test_df.pop('id')

0        200000
1        200001
2        200002
3        200003
4        200004
          ...  
99995    299995
99996    299996
99997    299997
99998    299998
99999    299999
Name: id, Length: 100000, dtype: int64

In [5]:
target = [int(i[-1]) - 1 for i in target]
target = np.asarray(target)

In [6]:
df = StandardScaler().fit_transform(df)
test_df = StandardScaler().fit_transform(test_df)

In [7]:
input_shape = df.shape[1]

In [8]:
batch_size = 4

In [9]:
for i, j  in enumerate(np.unique(target)):
    print(f'Count for Class{i}: {list(target).count(j)}')

Count for Class0: 9118
Count for Class1: 24431
Count for Class2: 14798
Count for Class3: 4704
Count for Class4: 3064
Count for Class5: 51811
Count for Class6: 14769
Count for Class7: 51763
Count for Class8: 25542


In [12]:
train = df[0:int(len(df)*.65)]
y_train = target[0:int(len(df)*.65)]
val_x = df[int(len(df)*.65):] 
val_y = target[int(len(df)*.65):]

In [13]:
def Model():
    x = Sequential()
    x.add(layers.Dense(128, activation = 'relu', input_shape = (input_shape,)))
    x.add(layers.Dense(256, activation = 'relu'))
    x.add(layers.Dense(512, activation = 'relu'))
    x.add(layers.Dense(1028, activation = 'relu'))
    x.add(layers.Dense(512, activation = 'relu'))
    x.add(layers.Dense(256, activation = 'relu'))
    x.add(layers.Dropout(.4))
    x.add(layers.Dense(128, activation = 'relu'))
    x.add(layers.Dense(64, activation = 'relu'))
    x.add(layers.Dense(9, activation = 'relu'))
    return x

In [14]:
model = Model()

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               9728      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 1028)              527364    
_________________________________________________________________
dense_4 (Dense)              (None, 512)               526848    
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0

In [16]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(df, target, batch_size, epochs = 5, validation_data= (val_x, val_y)) # try weighting classes based on representation in training data

Epoch 1/5
50000/50000 [==============================] - 847s 17ms/step - loss: 2.3058 - val_loss: 1.8444
Epoch 2/5
50000/50000 [==============================] - ETA: 0s - loss: 2.2836- ETA: 1

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
preds = probability_model.predict(test_df)

In [ ]:
preds_copy = preds

In [ ]:
np.max(preds_copy)

In [ ]:
preds_copy[1]

In [ ]:
preds_df = pd.DataFrame(preds_copy, columns = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9'])

In [ ]:
preds_df['id'] = range(200000,300000)
preds_df = preds_df.set_index(preds_df['id'])
preds_df = preds_df.drop('id', axis = 1)
preds_df.head()

In [ ]:
preds_df.to_csv('fourth_NN.csv')